In [1]:
from dmd_control import dmd
import imageio
import numpy as np
import time
from glob import glob
import os
from datetime import datetime

In [2]:
XP_FOLDER = r'D:\Data\Or42b_57C10__Chronos-mVenus_rGECO'
TARGET_FOLDER = XP_FOLDER + r'\2_488_20230829_164410'
GENERAL_PARAMETERS = r'C:\\Users\\Light Sheet User\\Desktop\\DMD_parameters.pickled'

In [3]:
mirrors = dmd(XP_FOLDER, TARGET_FOLDER, GENERAL_PARAMETERS)

Loading library: C:/Program Files/ALP-4.3/ALP-4.3 API/x64/alp4395.dll
DMD found, resolution = 1920 x 1200.
DMD patterns folder already exists.


##### Calibration procedure:
- project the *calibration_mask*
- save the image through the Labview code
- run the *find_affine* method
The affine transorm parameters are save or future use.

In [4]:
mirrors.project(mirrors.calibration_mask)

Save the calibration image. Usually choosing the same folder as **mirrors.parameters['params_folder']** is a good choice to keep things tidy.
Then plug it in the following method and run it to compute the afffine transform parameters.

In [ ]:
calibration_image = r'C:\Users\Light Sheet User\Documents\GitHub\DMD\camera_DMD_calibration.png'
mirrors.find_affine(calibration_image)

In [ ]:
mirrors.parameters['params_folder']

#### ROI selection
Use the following method to select ROIs to be stimulated by the DMD.
It is supposed to read sqaure ROIs, where you start drawing from top left corner to bottom right corner of the ROI. Different geometries are not implemented and may result in random or no pattern at all.

In [26]:
mirrors.select_ROIs()

In [27]:
mirrors.import_N_rois(exp='2_test')

Found 2 ROI(s)
Target ROI(s) folder already exists.
offsets [[0. 0.]]
offsets [[0. 0.]]


In [42]:
img1 = imageio.imread(mirrors.save_dir + '\\mask_1.png')
img2 = imageio.imread(mirrors.save_dir + '\\mask_2.png')

def sum_patterns(patterns):
    total = np.ones((patterns[0].shape), dtype=np.uint16)
    for pattern in patterns:
        total *= pattern
        total[total[:, :] > 1] = 255
    total = np.asarray(total, dtype=np.uint8)
    return total

tot = sum_patterns([img1, img2])
mirrors.project(tot)

C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_16472\150084632.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imageio.imread(mirrors.save_dir + '\\mask_1.png')
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_16472\150084632.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img2 = imageio.imread(mirrors.save_dir + '\\mask_2.png')


### PROTOCOL:
Load the images you want to use, and input how long each image should be displayed.

In [23]:
img1 = imageio.imread(mirrors.save_dir + '\\mask_1.png')

img_seq = [img1, mirrors.dark, img1, mirrors.dark]
img_seq_names = ['od1', 'dark', 'od1', 'dark'] # for saving later
img_times = [5, 5, 5, 10] # in seconds
reps = 3 # whole sequence repetition
start = time.time()
mirrors.sequence_of_single_images(img_seq, img_times, reps)
mirrors.idle()

# SAVE THE PROTOCOL
images_folder = mirrors.xp_folder + r'\2_2_20230829_171405'
filenames_camera_0 = r'\SPC00_TM*_ANG000_CM0_CHN00_PH0.stack'
all_volumes = sorted(glob(images_folder + filenames_camera_0))
for num, volume in enumerate(all_volumes):
    current = os.path.getctime(volume)
    if current - start > 0:
        break

np.savez(images_folder + "triggers_numpy", 
         TM=num, images=img_seq_names, durations=img_times)

# HOW TO READ IT:
# a = np.load(images_folder + "triggers_numpy.npz")
# lst = a.files
# for item in lst:
#     print(item)
#     print(a[item])

C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_16472\1259979119.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imageio.imread(mirrors.save_dir + '\\mask_1.png')


In [25]:
a = np.load(images_folder + "triggers_numpy.npz")
lst = a.files
for item in lst:
    print(item)
    print(type(item))
    print(a[item])

TM
<class 'str'>
558
images
<class 'str'>
['od1' 'dark' 'od1' 'dark']
durations
<class 'str'>
[ 5  5  5 10]


Close the DMD when done.

In [ ]:
mirrors.close()